In [5]:
import pandas as pd
from datetime import datetime

from analysis.tax import AutoTax

t = AutoTax()
t.ticker_detail('SRS')

SRS


,,Volume,TradePrice,EffectivePrice,Brokerage,Pro-rate_brokerage
Type,Date,,,,,
Buys,2018-05-15,30333.0,0.033,0.033330,0.0,10.0
Sells,2018-07-19,-30333.0,0.038,0.037342,20.0,NaN
